## Import du module Pandas et lecture des bases de données (numéro 1 : réponses à un sondage de salariés, numéro 2 : offres d'emploi)

In [19]:
# Import of the useful libraries
import pandas as pd
import sqlite3

# Import of sys so that Python recognizes the folders as modules. Besides, we have created empty __init__.py files to use our files as modules too
import sys
sys.path.insert(0, "/home/apprenant/Documents/Brief-4-Rachid-Karbiche")

# Import of the objects of our py files
from src.d02_intermediate.integrate_data import save_to_sql
from src.d01_data.create_database import conn, cur

# Reading of our first database Salary Survey Responses
survey_path = '../Data/01_raw/2020_Data_Professional_Salary_Survey_Responses.xlsx'
survey_data = pd.read_excel(survey_path, engine='openpyxl', skiprows=3)

# Reading of our second database DataAnalyst
job_applying_path = '../Data/01_raw/DataAnalyst.csv'
job_applying_data = pd.read_csv(job_applying_path)


## 1. BDD 1 : Examen de la base de données et suppression des colonnes inutiles pour le brief

Pour cette première phase, analysons la structure de notre base de données

In [20]:
print(survey_data.head())
print(survey_data.columns)
print(survey_data.shape)

   Survey Year               Timestamp  SalaryUSD        Country PostalCode  \
0         2020 2020-01-04 18:50:34.328   115000.0  United States         03   
1         2020 2020-01-04 10:43:01.821   100000.0  United States        NaN   
2         2020 2020-01-04 09:51:45.885   100000.0          Spain      28046   
3         2020 2020-01-04 01:08:53.605    70000.0  United States      94133   
4         2020 2020-01-03 15:28:54.163   110000.0  United States      95354   

        PrimaryDatabase  YearsWithThisDatabase  \
0  Microsoft SQL Server                     15   
1                 Other                      6   
2  Microsoft SQL Server                      2   
3  Microsoft SQL Server                      3   
4                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee   
1                                      MySQL/MariaDB  Full time employee

La base de données est composée de 8 627 entrées pour 31 colonnes. Parmi elles, certaines ne seront pas utiles dans l'étude de la base de données.

### Quelles colonnes conserver ?

In [21]:
survey_data = survey_data[['SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam',
          'PopulationOfLargestCityWithin20Miles', 'CareerPlansThisYear',
          'Gender']]
print(survey_data.head())

   SalaryUSD        Country PostalCode    EmploymentStatus  \
0   115000.0  United States         03  Full time employee   
1   100000.0  United States        NaN  Full time employee   
2   100000.0          Spain      28046  Full time employee   
3    70000.0  United States      94133  Full time employee   
4   110000.0  United States      95354  Full time employee   

                                            JobTitle  YearsWithThisTypeOfJob  \
0  DBA (Production Focus - build & troubleshoot s...                       5   
1                  Developer: App code (C#, JS, etc)                       6   
2                                              Other                       3   
3                                            Analyst                       3   
4  DBA (General - splits time evenly between writ...                      25   

                                    HowManyCompanies OtherPeopleOnYourTeam  \
0  1 (this is the only company where I've had thi...                

Dans le cadre de mon travail, j'ai choisi de conserver onze d'entre elles. J'aurais pu en conserver davantage, notamment dans le cadre de la question sur le plan de carrière de notre rêve américain. D'ailleurs, pour celle-ci, j'ai gardé les colonnes EmploymentStatus, CareerPlansThisYear et PopulationOfLargestCityWithin20Miles.

## 2. BDD1 : vérification des données manquantes dans la base de données

In [22]:
print(survey_data.isnull().sum())

postalcode_missing_values = survey_data.PostalCode.isnull().sum()
total_values = survey_data.shape[0]
part_postalcode_missing_values = (postalcode_missing_values / total_values) * 100
print(part_postalcode_missing_values, '% de données manquantes')

SalaryUSD                                  0
Country                                    0
PostalCode                              1390
EmploymentStatus                           0
JobTitle                                   0
YearsWithThisTypeOfJob                     0
HowManyCompanies                           0
OtherPeopleOnYourTeam                      0
PopulationOfLargestCityWithin20Miles       0
CareerPlansThisYear                        0
Gender                                     0
dtype: int64
16.112205865306596 % de données manquantes


Comme spécifié plus haut, le nombre d'entrées dans la première base de données est de 8 627. Ici, PostalCode détient 1 390 entrées manquantes, ce qui représente un peu plus de 16 % des entrées totales. Cette donnée n'est pas la plus importante de la base et sa manipulation n'en sera que ponctuelle donc il n'est pas nécessaire de toucher à ces valeurs manquantes.

## Qu'en est-il de ces codes postaux manquants ?

Il serait difficile d'uniformiser les codes postaux car il n'existe pas un code postal universel. De ce fait, pas besoin d'y toucher, surtout qu'elle n'est pas prépondérante dans l'étude de la base de données.

## 3. BDD1 : Le traitement des dates dans la base de données

In [23]:
print(survey_data.dtypes)

SalaryUSD                               float64
Country                                  object
PostalCode                               object
EmploymentStatus                         object
JobTitle                                 object
YearsWithThisTypeOfJob                    int64
HowManyCompanies                         object
OtherPeopleOnYourTeam                    object
PopulationOfLargestCityWithin20Miles     object
CareerPlansThisYear                      object
Gender                                   object
dtype: object


Aucune donnée de type "date" présente dans la base de données donc passons à la suite !

## 4. BDD1 : Gérons les doublons !

In [24]:
print(survey_data.duplicated().value_counts())
print(survey_data[survey_data.duplicated(keep=False)])

survey_data = survey_data.drop_duplicates(keep='first')
print(survey_data.duplicated().value_counts())

False    8479
True      148
dtype: int64
      SalaryUSD        Country PostalCode    EmploymentStatus  \
52     148000.0  United States      60642  Full time employee   
53     148000.0  United States      60642  Full time employee   
62     100000.0         Canada        NaN  Full time employee   
64     100000.0         Canada        NaN  Full time employee   
144     71500.0  United States      48192  Full time employee   
...         ...            ...        ...                 ...   
8523    95000.0  United States  Not Asked  Full time employee   
8562   105000.0  United States  Not Asked  Full time employee   
8595    52000.0  United States  Not Asked  Full time employee   
8611    96000.0  United States  Not Asked  Full time employee   
8612    85000.0  United States  Not Asked  Full time employee   

              JobTitle  YearsWithThisTypeOfJob  \
52      Data Scientist                       7   
53      Data Scientist                       7   
62             Analyst      

La base de données est composée de plusieurs doublons, 148 précisément. En examinant la base de données, on remarque que seul le timestamp permet de différencier les doublons entre eux. On peut donc relever ceux-ci en supprimant la colonne Timestamp. L'enregistrement pour ces doublons est seulement différencié de quelques secondes. On peut donc les enlever et ne garder qu'un exemplaire de chaque.

## 5. BDD1 : Vérifions la consistence des données !

Le but est de relever des inconsistances dans les noms des données. 

In [25]:
print(sorted(survey_data.Country.unique()))
print("----------------------------------")
survey_data = survey_data.loc[survey_data['Country'] == 'United States']
print(survey_data)

['Albania', 'Anguilla', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Bahrain', 'Belarus', 'Belgium', 'Bermuda', 'Bolivia', 'Brazil', 'Bulgaria', 'Canada', 'Cayman Islands', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guernsey', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Jersey', 'Jordan', 'Kenya', 'Latvia', 'Lithuania', 'Luxembourg', 'Macedonia', 'Malaysia', 'Maldives', 'Malta', 'Mexico', 'Moldova', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua', 'Norway', 'Pakistan', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Romania', 'Russia', 'Saudi Arabia', 'Serbia and Montenegro', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'Spain', 'Swaziland', 'Sweden', 'Switzerland', 'Syria', 'Taiwan', 'Thailand', 'Turkey', 'Ugan

Dans le cadre du brief, les questions concernent exclusivement les Etats-Unis donc on peut enlever les lignes qui concernent les autres pays. Au lieu de 8 627 lignes, nous n'en avons plus que 5 542.

In [26]:
print(sorted(survey_data.EmploymentStatus.unique()))
print("----------------------------------")
print(sorted(survey_data.JobTitle.unique()))
print("------------------------------------")
print(survey_data.JobTitle.value_counts())
print("----------------------------------")
print(sorted(survey_data.PopulationOfLargestCityWithin20Miles.unique()))
print("----------------------------------")
print(sorted(survey_data.Gender.unique()))


['Full time employee', 'Full time employee of a consulting/contracting company', 'Independent consultant, contractor, freelancer,  or company owner', 'Independent or freelancer or company owner', 'Part time']
----------------------------------
['Analyst', 'Analytics consultant', 'Architect', 'Consultant', 'DBA', 'DBA (Development Focus - tunes queries, indexes, does deployments)', 'DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)', 'DBA (Production Focus - build & troubleshoot servers, HA/DR)', 'Data Scientist', 'Database Specialist', 'DevOps, Sr Software Engineer DBA', 'Developer: App code (C#, JS, etc)', 'Developer: Business Intelligence (SSRS, PowerBI, etc)', 'Developer: T-SQL', 'Engineer', 'Manager', 'Other', 'Principal database engineer', 'Sr Consultant ', 'Systems Administrator', 'Technician ']
------------------------------------
DBA (General - splits time evenly between writing & tuning queries AND building & troubleshoo

En voyant les valeurs uniques de différentes colonnes, on remarque que les données semblent consistantes sauf du côté de nos métiers. En effet, plusieurs métiers ont pour titre DBA avec de nombreuses précisions derrière. De plus, certains métiers sont sous-représentés en nombre. On va donc faire deux choses : regrouper les métiers sous-représentés en une catégorie "Other" et regrouper aussi entre eux les métiers qui commencent par DBA.

In [27]:
x = survey_data.JobTitle.value_counts()
survey_data.loc[survey_data.JobTitle.isin(x[x <= 5].index), "JobTitle"] = "Other"
print(survey_data.JobTitle.value_counts())
print("-------------------------------------")
survey_data.JobTitle = survey_data.JobTitle.apply(lambda x: 'DBA' if 'DBA' in x else x)
print(survey_data)

DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)    1007
DBA                                                                                                            882
DBA (Production Focus - build & troubleshoot servers, HA/DR)                                                   579
Developer: T-SQL                                                                                               529
Architect                                                                                                      460
Manager                                                                                                        378
Developer: Business Intelligence (SSRS, PowerBI, etc)                                                          359
Developer: App code (C#, JS, etc)                                                                              350
Analyst                                                                         

## 6. BDD1 : Des données aberrantes ?

La première colonne à observer est celle des salaires. En effet, certains salaires sont très bas mais nous n'avons pas beaucoup plus d'informations. De ce fait, j'ai choisi d'enlever tous les salaires inférieurs à 10 000 dollars. En faisant les moyennes de toutes les valeurs avant retrait, on tombe sur une moyenne de 108 767 dollars. Après retrait, la moyenne est de 108 905 dollars donc ces valeurs n'ont pas trop de conséquence sur cette statistique.

In [28]:
print(sorted(survey_data.SalaryUSD.unique()))
print("------------------------------------")
print(survey_data.SalaryUSD.mean())
print("------------------------------------")
selected_salaries = survey_data[survey_data['SalaryUSD'] > 10000]
print(selected_salaries)
print("------------------------------------")
print(selected_salaries.SalaryUSD.mean())

[0.0, 92.27, 125.0, 130.0, 135.0, 144.0, 150.0, 11100.0, 11500.0, 11800.0, 12300.0, 24000.0, 25000.0, 30000.0, 31200.0, 32000.0, 35000.0, 35800.0, 36000.0, 37400.0, 37500.0, 38000.0, 38500.0, 39000.0, 40000.0, 42000.0, 42500.0, 43000.0, 44000.0, 45000.0, 45760.0, 46000.0, 46350.0, 46820.0, 47000.0, 47240.0, 47500.0, 47840.0, 48000.0, 48500.0, 49000.0, 49260.0, 49500.0, 49900.0, 50000.0, 50085.0, 50750.0, 51000.0, 51700.0, 51800.0, 52000.0, 52500.0, 53000.0, 53040.0, 53500.0, 54000.0, 55000.0, 55500.0, 55600.0, 56000.0, 56500.0, 56600.0, 56700.0, 56800.0, 57000.0, 57200.0, 57300.0, 57400.0, 57500.0, 58000.0, 58500.0, 58775.0, 59000.0, 59009.0, 59100.0, 59700.0, 60000.0, 60090.0, 60166.0, 60500.0, 60600.0, 60900.0, 61000.0, 61200.0, 61500.0, 61773.0, 61800.0, 61900.0, 62000.0, 62400.0, 62500.0, 62661.0, 62700.0, 63000.0, 63300.0, 63330.0, 63344.0, 63400.0, 63500.0, 63600.0, 64000.0, 64477.0, 64480.0, 64750.0, 64775.0, 65000.0, 65240.0, 65500.0, 65543.0, 66000.0, 66050.0, 66500.0, 66625.0

## Intégration des données dans les tables de la base de données

In [29]:
save_to_sql(survey_data, "Responses", conn)

## 1. BDD 2 : Examen de la base de données et suppression des colonnes inutiles pour le brief

Pour cette première phase, analysons la structure de notre base de données

In [30]:
print(job_applying_data.head())
print(job_applying_data.columns)
print(job_applying_data.shape)

   Unnamed: 0                                          Job Title  \
0           0  Data Analyst, Center on Immigration and Justic...   
1           1                               Quality Data Analyst   
2           2  Senior Data Analyst, Insights & Analytics Team...   
3           3                                       Data Analyst   
4           4                             Reporting Data Analyst   

              Salary Estimate  \
0  $37K-$66K (Glassdoor est.)   
1  $37K-$66K (Glassdoor est.)   
2  $37K-$66K (Glassdoor est.)   
3  $37K-$66K (Glassdoor est.)   
4  $37K-$66K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Are you eager to roll up your sleeves and harn...     3.2   
1  Overview\n\nProvides analytical and technical ...     3.8   
2  We’re looking for a Senior Data Analyst who ha...     3.4   
3  Requisition NumberRR-0001939\nRemote:Yes\nWe c...     4.1   
4  ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...     3.9   

       

La base de données est composée de 2 253 entrées pour 16 colonnes. Parmi elles, certaines ne seront pas utiles dans l'étude de la base de données.

In [31]:
job_applying_data = job_applying_data[['Job Title', 'Salary Estimate', 'Location']]
print(job_applying_data.head())

                                           Job Title  \
0  Data Analyst, Center on Immigration and Justic...   
1                               Quality Data Analyst   
2  Senior Data Analyst, Insights & Analytics Team...   
3                                       Data Analyst   
4                             Reporting Data Analyst   

              Salary Estimate      Location  
0  $37K-$66K (Glassdoor est.)  New York, NY  
1  $37K-$66K (Glassdoor est.)  New York, NY  
2  $37K-$66K (Glassdoor est.)  New York, NY  
3  $37K-$66K (Glassdoor est.)  New York, NY  
4  $37K-$66K (Glassdoor est.)  New York, NY  


Dans le cadre de mon travail, j'ai choisi de conserver seulement trois colonnes. En effet, pour répondre aux questions, ces colonnes choisies seront utiles. POur la colonne Location, elle pourrait être intéressante pour la question du plan de carrière.

## 2. BDD2 : vérification des données manquantes dans la base de données

In [32]:
print(job_applying_data.isnull().sum())

Job Title          0
Salary Estimate    0
Location           0
dtype: int64


On constate ici qu'il n'y a pas de données manquantes dans la base de données.

## 3. BDD2 : Le traitement des dates dans la base de données

In [33]:
print(job_applying_data.dtypes)

Job Title          object
Salary Estimate    object
Location           object
dtype: object


Aucune date à l'horizon donc on peut poursuivre notre data cleaning

## 4. BDD2 : Gérons les doublons !

In [34]:
print(job_applying_data.duplicated().value_counts())
print(job_applying_data[job_applying_data.duplicated(keep=False)])

job_applying_data = job_applying_data.drop_duplicates(keep='first')
print(job_applying_data.duplicated().value_counts())

False    1895
True      358
dtype: int64
                         Job Title              Salary Estimate  \
3                     Data Analyst   $37K-$66K (Glassdoor est.)   
5                     Data Analyst   $37K-$66K (Glassdoor est.)   
7             Data Science Analyst   $37K-$66K (Glassdoor est.)   
8                     Data Analyst   $37K-$66K (Glassdoor est.)   
10                    Data Analyst   $37K-$66K (Glassdoor est.)   
...                            ...                          ...   
2227  Data Base Programmer/Analyst  $57K-$100K (Glassdoor est.)   
2231               Sr Data Analyst  $57K-$100K (Glassdoor est.)   
2235  Data Base Programmer/Analyst  $57K-$100K (Glassdoor est.)   
2238  Senior Contract Data Analyst  $78K-$104K (Glassdoor est.)   
2239  Senior Contract Data Analyst  $78K-$104K (Glassdoor est.)   

            Location  
3       New York, NY  
5       New York, NY  
7       New York, NY  
8       New York, NY  
10      New York, NY  
...             

Cas particulier ici : nous avons 3 colonnes sur lesquels travailler. Or, nous repérons des doublons sur ces seules trois colonnes, ce qui est plutôt normal. En vérifiant en amont sur la base de données brutes, il n'y a en réalité aucun doublon. Dans ce cas-là, on peut bien sûr se permettre de supprimer les doublons car nous n'avons pas beaucoup de colonnes. Cela allège notre base au moins et au final, cela n'aurait rien changé, suppression ou pas.

## 5. BDD2 : Vérifions la consistence des données !

In [35]:
print(sorted(job_applying_data.Location.unique()))
print("---------------------------------------")
print(job_applying_data['Job Title'].unique())
print("---------------------------------------")
print(sorted(job_applying_data['Salary Estimate'].unique()))

['Addison, TX', 'Alachua, FL', 'Alameda, CA', 'Alhambra, CA', 'Allegheny West, PA', 'Allen, TX', 'American Fork, UT', 'Anaheim, CA', 'Arcadia, CA', 'Arlington Heights, IL', 'Arlington, TX', 'Athens, GA', 'Aurora, CO', 'Austin, TX', 'Azusa, CA', 'Beech Grove, IN', 'Bellevue, WA', 'Bensalem, PA', 'Berkeley Heights, NJ', 'Berkeley, CA', 'Berwyn, PA', 'Beverly Hills, CA', 'Blue Bell, PA', 'Boothwyn, PA', 'Boulder, CO', 'Brea, CA', 'Bridgeview, IL', 'Broadview, IL', 'Bronx, NY', 'Brooklyn, NY', 'Broomfield, CO', 'Burbank, CA', 'Burlingame, CA', 'Burlingame, KS', 'Burr Ridge, IL', 'Camden, NJ', 'Campbell, CA', 'Carmel, IN', 'Carrollton, TX', 'Carson, CA', 'Cedar Park, TX', 'Centennial, CO', 'Cerritos, CA', 'Chandler, AZ', 'Charlotte, NC', 'Chesapeake, VA', 'Chester Township, PA', 'Chicago, IL', 'City of Industry, CA', 'Columbus, OH', 'Conshohocken, PA', 'Coppell, TX', 'Culver City, CA', 'Cupertino, CA', 'DC Ranch, AZ', 'Dallas, TX', 'Daly City, CA', 'Deerfield, IL', 'Denver, CO', 'Des Plaine

Rien de particulier à signaler de ce côté. On peut passer à la suite

## 6. BDD2 : Des données aberrantes ?

## Intégration des données dans les tables de la base de données

In [36]:
save_to_sql(job_applying_data, "Job_Adverts", conn)

/home/apprenant/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
